# Deep Hedging in Incomplete Markets — GBM & Heston

**MSc Thesis Experiment Runner**

Runs the full deep hedging pipeline under two market dynamics:
- **GBM** (constant volatility, calibrated to S&P 500)
- **Heston** (stochastic volatility, calibrated to S&P 500)

**Before running:**
1. **Runtime → Change runtime type → A100 GPU** (Pro+ recommended)
2. Click **Connect**

**Runtime protection:**
- Outputs are saved to Google Drive after each stage so nothing is lost on disconnect
- GBM and Heston run as separate cells (~2-4h each) — safer than one long run
- Keep-alive prevents idle timeout

In [ ]:
# Cell 1: Mount Google Drive + Clone repo + Install deps
from google.colab import drive
drive.mount("/content/drive")

# Persistent output directory on Drive
DRIVE_OUT = "/content/drive/MyDrive/deep_hedging_outputs"
!mkdir -p "{DRIVE_OUT}"

!git clone https://github.com/thabangTheActuaryCoder/deep-hedging-thesis.git
%cd deep-hedging-thesis
!pip install -q torch numpy matplotlib optuna

import torch
print(f"\nPython: {__import__('sys').version}")
print(f"PyTorch: {torch.__version__}")
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device: {torch.cuda.get_device_name(0)}")
    mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"Memory: {mem:.1f} GB")
print(f"\nDrive output dir: {DRIVE_OUT}")

In [ ]:
# Cell 2: Keep-alive (prevents idle disconnect — run this, then move on)
# This runs in the background; you only need to run it once per session.
import IPython
IPython.display.display(IPython.display.Javascript('''
function keepAlive() {
    google.colab.kernel.invokeFunction("shell", ["echo alive"], {});
    setTimeout(keepAlive, 60000);
}
keepAlive();
'''))
print("Keep-alive active (pings every 60s)")

In [ ]:
# Cell 3: Sanity check — all tests should pass
!python -m pytest tests/test_validation.py -v

In [ ]:
# Cell 4 (QUICK TEST): ~10 min, verifies both pipelines work
!python run_experiment.py --quick --market_model both \
    --output_dir /content/drive/MyDrive/deep_hedging_outputs

In [ ]:
# Cell 5 (FULL RUN): both GBM + Heston, 100k paths
#
# Outputs go DIRECTLY to Google Drive, so if the runtime disconnects:
#   1. Reconnect, re-run Cells 1 + 2
#   2. Re-run this cell — Stage 1 (Optuna) picks up from cache on Drive,
#      only the interrupted Stage 2 (seed robustness) reruns.
#
# Expected time: ~4-8 hours on A100

!python run_experiment.py \
    --paths 100000 \
    --N 200 \
    --epochs 1000 \
    --patience 15 \
    --batch_size 2048 \
    --n_trials 60 \
    --seeds 0 1 2 3 4 \
    --substeps 0 5 10 \
    --market_model both \
    --output_dir /content/drive/MyDrive/deep_hedging_outputs

In [ ]:
# Cell 6: Preview GBM validation plots
from IPython.display import Image, display
import glob

OUT = "/content/drive/MyDrive/deep_hedging_outputs"
print("=== GBM Validation Plots ===")
for img in sorted(glob.glob(f"{OUT}/gbm/plots_val/*.png")):
    print(f"\n--- {img.split('/')[-1]} ---")
    display(Image(filename=img, width=700))

In [ ]:
# Cell 7: Preview Heston validation plots
from IPython.display import Image, display
import glob

OUT = "/content/drive/MyDrive/deep_hedging_outputs"
print("=== Heston Validation Plots ===")
for img in sorted(glob.glob(f"{OUT}/heston/plots_val/*.png")):
    print(f"\n--- {img.split('/')[-1]} ---")
    display(Image(filename=img, width=700))

In [ ]:
# Cell 8: Heston stochastic volatility diagnostic plots
from IPython.display import Image, display
import glob

OUT = "/content/drive/MyDrive/deep_hedging_outputs"
print("=== Heston Stochastic Volatility Diagnostics ===")
for img in sorted(glob.glob(f"{OUT}/heston/plots_heston/*.png")):
    print(f"\n--- {img.split('/')[-1]} ---")
    display(Image(filename=img, width=700))

In [ ]:
# Cell 9: GBM vs Heston comparison plots
from IPython.display import Image, display
import glob

OUT = "/content/drive/MyDrive/deep_hedging_outputs"
print("=== GBM vs Heston Comparison ===")
for img in sorted(glob.glob(f"{OUT}/comparison/*.png")):
    print(f"\n--- {img.split('/')[-1]} ---")
    display(Image(filename=img, width=700))

In [ ]:
# Cell 10: 3D delta surface plots
from IPython.display import Image, display
import glob

OUT = "/content/drive/MyDrive/deep_hedging_outputs"
for label, pattern in [("GBM", f"{OUT}/gbm/plots_3d/*.png"),
                       ("Heston", f"{OUT}/heston/plots_3d/*.png")]:
    imgs = sorted(glob.glob(pattern))
    if imgs:
        print(f"\n=== {label} 3D Delta Surfaces ===")
        for img in imgs:
            print(f"\n--- {img.split('/')[-1]} ---")
            display(Image(filename=img, width=700))

In [ ]:
# Cell 11: Show validation metrics (both market models)
import json, os

OUT = "/content/drive/MyDrive/deep_hedging_outputs"

for market in ["gbm", "heston"]:
    path = f"{OUT}/{market}/val_metrics.json"
    if not os.path.exists(path):
        continue
    with open(path) as f:
        metrics = json.load(f)
    print(f"\n{'='*50}")
    print(f"  {market.upper()} — Best model: {metrics['best_model']}")
    print(f"{'='*50}")
    for model, agg in metrics["aggregated_val_metrics"].items():
        cvar = agg["CVaR95_shortfall"]
        mse = agg["MSE"]
        print(f"  {model:6s}  CVaR95 = {cvar['mean']:.6f} +/- {cvar['std']:.6f}  "
              f"MSE = {mse['mean']:.6f} +/- {mse['std']:.6f}")

summary_path = f"{OUT}/metrics_summary.json"
if os.path.exists(summary_path):
    with open(summary_path) as f:
        combined = json.load(f)
    print(f"\n{'='*50}")
    print("  COMBINED SUMMARY")
    print(f"{'='*50}")
    for market, agg in combined.items():
        print(f"\n  [{market.upper()}]")
        for model, m in agg.items():
            cvar = m.get("CVaR95_shortfall", {})
            if isinstance(cvar, dict):
                print(f"    {model:6s}  CVaR95 = {cvar.get('mean',0):.6f} +/- {cvar.get('std',0):.6f}")

## Replot (optional)

If you want to adjust figure dimensions, colors, grids, or fonts without
rerunning the experiment, edit the `STYLE` dict in `replot.py` and rerun
Cell 13 below. The data was saved during the experiment.

In [ ]:
# Cell 13: Regenerate comparison plots from saved data (edit STYLE in replot.py first)
OUT = "/content/drive/MyDrive/deep_hedging_outputs"

!python replot.py --data "{OUT}/comparison/comparison_data.pt" \
                  --metrics "{OUT}/metrics_summary.json" \
                  --out "{OUT}/comparison"

from IPython.display import Image, display
import glob
for img in sorted(glob.glob(f"{OUT}/comparison/*.png")):
    print(f"\n--- {img.split('/')[-1]} ---")
    display(Image(filename=img, width=700))

In [ ]:
# Cell 14: Download outputs as zip (optional — they're already on Drive)
import shutil
from google.colab import files

OUT = "/content/drive/MyDrive/deep_hedging_outputs"
shutil.make_archive("/content/deep_hedging_outputs", "zip", OUT)
files.download("/content/deep_hedging_outputs.zip")